[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/ProjIntegrador_PO_IA/blob/main/kmeans_knn.ipynb)


# Aplicações de Pesquisa Operacional e Inteligência Artificial



# Introdução

Nesta aula, veremos dois procedimentos baseados em distâncias. O primeiro, k-médias, é um algoritmo de **clustering**, classificado como um procedimentos de aprendizado **não supervisionado**. Já o segundo procedimento, o k-NN, consiste em um algoritmo de aprendizado **supervisionado** que efetua tarefas de **classificação**. 

- K-médias - https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
- K-NN - https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

## Conjunto de dados


Trabalharemos com um conjunto de dados do Legathum Prosperity Index que avalia países com base em 12 indicadores socioeconômicos. Os dados estão armazenados em uma planilha Excel. Para realizar a leitura, utilizaremos a biblioteca Pandas.

In [ ]:
import pandas as pd

In [ ]:
# Leitura dos dados
url = 'https://raw.githubusercontent.com/diogoflim/ProjIntegrador_PO_IA/main/Dados/LegathumProsperityIndex.csv'

df = pd.read_csv(url, index_col=0, sep = ';')
df

Podemos inicialmente explorar o conjunto de dados com algumas estatísticas. Utilizaremos inicialmente o método describe. Em seguida, Vamos plotar alguns histogramas.

In [ ]:
df.describe()

In [ ]:
# Vamos utilizar a biblioteca referência em python para plotar gráficos

import matplotlib.pyplot as plt

In [ ]:
plt.figure(1 , figsize = (6 , 8))
n = 0 
for atributo in ['Economic Quality',	'Education',	'Enterprise Conditions']:
    n += 1
    plt.subplot(3 , 1 , n)
    plt.subplots_adjust(hspace = 1 , wspace = 1)
    plt.hist(df[atributo])
    plt.title('Histograma do atributo {}'.format(atributo))

plt.show()

In [ ]:
# Exercício: Repita o procedimento acima escolhendo outros atributos

## Prodecimentos de Normalização

Em algoritmos baseados em distância, assim como em outros modelos de Aprendizado de Máquina, é comum efetuar algumas etapas de pré-processamento de dados. Uma dessas etapas é a normalização da matriz de atributos.


A normalização de dados refere-se ao processo de escalar os atributos de um conjunto de dados de forma que eles estejam dentro de um intervalo específico, como [0, 1] ou [-1, 1]. 

Isso pode ser feito de várias maneiras, e o scikit-learn fornece duas abordagens comuns: 
- Min-Max Scaling
- Z-score Standardization.


Min-Max Scaling, também conhecido como normalização intervalar [0, 1], é uma técnica que redimensiona os valores de um atributo para que fiquem dentro do intervalo especificado. 

A fórmula para Min-Max Scaling é dada por:

$$X_{\text{scaled}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}$$

Onde:

- X: Valor original do atributo
- $X_{\text{min}}$: Valor mínimo do atributo no conjunto de dados
- $X_{\text{max}}$: Valor máximo do atributo no conjunto de dados


Por sua vez, a Z-score Standardization, também conhecida como padronização, transforma os valores de uma característica para que tenham uma média de 0 e um desvio padrão de 1. 

A fórmula para Z-score Standardization é dada por:

$$X_{\text{scaled}} = \frac{X - \mu}{\sigma}$$

Onde:
- X: Valor original do atributo
- $\mu$: Média do atributo no conjunto de dados
- $\sigma$:Desvio padrão do atributo no conjunto de dados


In [ ]:
# Importando os métodos
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# Instanciando os procedimentos
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Realizando as transformações
df_minmax = min_max_scaler.fit_transform(df)
df_standardized = standard_scaler.fit_transform(df)

# Passando os dados normalizados para DataFrames
df_minmax = pd.DataFrame(df_minmax, columns=df.columns, index = df.index)
df_standardized = pd.DataFrame(df_standardized, columns=df.columns, index = df.index)

In [ ]:
df_minmax

In [ ]:
df_standardized

## Aplicando o k-médias

Utilizaremos os dados normalizados com o procedimento intervalar. Para a aplicação do k-médias, basta utilizarmos o procedimento da biblioteca sklearn.

In [ ]:
from sklearn.cluster import KMeans

k = 3 # Faremos o teste com 3 clusters

# Instanciando o modelo K-Means
kmeans = KMeans(n_clusters=k, random_state=42)

# Ajustando o modelo aos dados normalizados
kmeans.fit(df_minmax)

# Adicionando os rótulos de cluster aos dados originais
df['Cluster Labels'] = kmeans.labels_

In [ ]:
df

É comum no uso do k-means que testemos qual a quantidade ideal de clusters. 

Para isso, procedimentos comuns são:
- Curva do cotovelo;
- Curva da silhueta;

Vejamos o funcionamento da curva do cotovelo, onde testaremos valores de $k$ de 1 a 10 e calcularemos a inércia para cada um. 

Em seguida, a curva do cotovelo será plotada para visualizarmos a tendência. 

O valor de k ideal será aquele em que a curva começa a se achatar, formando um "cotovelo". Assim, escolhemos um valor de k que corresponda ao ponto onde a inércia começa a diminuir a uma taxa menor.

In [ ]:
inertia = []
for k in range(1, 11):  # Teste valores de k de 1 a 10
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_minmax)
    inertia.append(kmeans.inertia_)

# Plote a curva do cotovelo
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), inertia, marker='o', linestyle='-', color='b')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inércia')
plt.title('Curva do Cotovelo para Escolha de k')
plt.grid(True)
plt.show()

## Aplicando o k-NN

Como o k-NN consiste em um procedimento de Aprendizado Supervisionado, trabalhamos com a presença de rótulos para os exemplos de treinamento.

Para efeitos ilustrativos, vamos aproveitar nesse momento os rótulos atribuídos na clusterização do k-médias. Lembre que adicionamos uma coluna com o rótulo o nosso DataFrame df.

In [ ]:
# Imprimindo as três últimas colunas
df.iloc[:, -3:]

Inicialmente, vamos separar o DataFrame em uma matriz de atributos "X" e um vetor de rótulos "y"

In [ ]:
import numpy as np

In [ ]:
X = np.array(df.drop(columns = ["Cluster Labels"]))
y = np.array(df["Cluster Labels"])

In [ ]:
#Matriz de Atributos
X

In [ ]:
# Vetor de rótulos
y

In [ ]:
# Normalizando a matriz de atributos.

X = MinMaxScaler().fit_transform(X)

## Amostragem


Ao aplicar um modelo de aprendizado de máquina, é importante que utilizemos alguma técnica de amostragem. Por exemplo, no tradicional procedimento holdout, separamos os dados em dois conjuntos:

- Conjunto de treinamento: onde treinaremos o algoritmo de aprendizado;
- Conjunto de teste: onde testaremos o modelo aprendido e verificaremos a performance.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Agora que os dados foram separados, vamos treinar nosso algoritmo nos dados de treinamento:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
k_vizinhos = 3 # Número de vizinhos que utilizaremos inicialmente
knn_classifier = KNeighborsClassifier(n_neighbors = k_vizinhos) #Instanciando o modelo
knn_classifier.fit(X_train, y_train) # Treinando o modelo

Agora, podemos aplicar o algoritmo treinado nos dados de teste para realizar previsões das classes!

In [ ]:
y_pred = knn_classifier.predict(X_test)

y_pred

Vamos medir a acurácia atingida pelo nosso modelo:

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acuracia = accuracy_score(y_test, y_pred)
print(f"A acurácia obtida no conjunto de teste foi: {acuracia:.4f}")

Nosso modelo previu corretamente todo o conjunto treinado, atingindo uma acurácia de 100%.

Perceba que isso não necessariamente ocorre. Pode ter sido ao acaso. Vamos refazer nosso experimento com uma outra semente aleatória na separação dos conjunto de treinamento e teste.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)


k_vizinhos = 3
knn_classifier = KNeighborsClassifier(n_neighbors = k_vizinhos)
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)
print(f"A acurácia obtida no conjunto de teste foi: {acuracia:.4f}")

**Agora nossa acurácia foi de apenas 94,12%**

## Validação Cruzada

Para minimizar os efeitos problemáticos da amostragem Holdout, podemos partir para uma outra técnica de amostragem. Vejamos a técnica cross validation.


1. Divisão dos dados: Inicialmente, o conjunto de dados é dividido em duas partes: um conjunto de treinamento e um conjunto de teste. O conjunto de treinamento é usado para treinar o modelo, enquanto o conjunto de teste é reservado para avaliar o desempenho do modelo.

2. Iteração: No procedimento de validação cruzada, essa divisão em treinamento e teste é repetida várias vezes, chamadas "fold" ou dobras. Cada vez, uma parte diferente do conjunto de dados é usada como conjunto de teste, enquanto o restante é usado como conjunto de treinamento. Por exemplo, em uma validação cruzada de 5-fold, o conjunto de dados é dividido em 5 partes iguais, e o processo é repetido 5 vezes.

3. Treinamento e Avaliação: Durante cada iteração, um modelo é treinado no conjunto de treinamento e avaliado no conjunto de teste. As métricas de desempenho, como a precisão, podem ser registradas para cada iteração.

4. Cálculo das Métricas Finais: Após todas as iterações, as métricas de desempenho (por exemplo, precisão média) são calculadas a partir das métricas registradas em cada dobra. Isso fornece uma estimativa mais confiável do desempenho do modelo em dados não vistos.

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# validação cruzada 5-fold
num_folds = 5
acuracias_obtidas = cross_val_score(knn_classifier, X, y, cv=num_folds)


for i, score in enumerate(acuracias_obtidas):
    print(f"Fold {i + 1}: Acurácia = {score:.4f}")

# Calculando a acurácia média
acuracia_media = np.mean(acuracias_obtidas)
print(f"Acurácia Média Obtida no Experimento: {acuracia_media:.4f}")

De maneira geral, o nosso procedimento obteve bons resuldados, com uma acurácia média de 97,02%.

É importante lembrar que nesse teste utilizamos rótulos vindos do procedimento k-means. Isso não é usual. Normalmente, os rótulos são considerados respostas, por exemplo, atribuídas por um especialista. 

Um exemplo seria o diagnóstico (rótulos) históricos atribuídos por médicos de um hospotal a pacientes que chegam com determinados sintomes (matriz de atributos). 